<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>COMP 555 Bioalgorithms Spring 2021</h1>
<h1 style="font-size: 200%;">Problem Set 2</h1>
</div>

---
You will need a the following sequence collection of gene promoter regions in which you will search for <a href="http://csbio.unc.edu/mcmillan/Comp555S21/motifs.fa" download="motifs.fa">transcription binding factor motifs</a>.

In [1]:
import itertools
import math
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def loadFasta(filename):
    """ Parses a classically formatted and possibly compressed
        FASTA file into two lists. One of headers and a second
        list of sequences. The ith index of each list correspond.
    """
    if filename.endswith('.gz'):
        fp = gzip.open(filename, 'r')
    else:
        fp = open(filename, 'r')
    # Split at headers
    data = fp.read().split('>')
    fp.close()
    # Ignore whatever appears before the first header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # Add an extra '+' to make string '1-referenced'
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

def ScanAndScoreMotif(DNA, motif):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if HammingDist < minHammingDist:
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
    return bestAlignment, totalDist

def MedianStringMotifSearch(DNA, k):
    # Consider all possible 4**k motifs
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for pattern in itertools.product('ACGT', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif)
        if dist < minHammingDist:
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

def ContainedMotifSearch(DNA, k):
    # Consider only motifs from the given DNA sequences
    motifSet = set()
    for seq in DNA:
        for i in range(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
    print('%d Motifs in our set' % len(motifSet))
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for motif in motifSet:
        align, dist = ScanAndScoreMotif(DNA, motif)
        if dist < minHammingDist:
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

---
**Problem 1:** In the cell below use the given <code>**MedianStringMotifSearch()**</code>, and <code>**ScanAndScoreMotif()**</code> functions to find an optimal 10-base motif pattern and its location in each of the 7 given promoter regions in <a href="http://csbio.unc.edu/mcmillan/Comp555S21/motifs.fa" download="motifs.fa">motifs.fa</a>. Based on this result, make a prediction of how long it would have taken to find an optimal 12-base motif.

In [2]:
hdr, dna = loadFasta('data/motifs.fa')
%time MedianStringMotifSearch(dna, 10)

Wall time: 2h 33min 12s


([875, 74, 114, 371, 6, 72, 190], 9, 'AGCCTGGGCA')

In [3]:
import datetime

def revPred(hrs, mins, s):
    s = (hrs*60*60) + (mins*60) + s
    s = s*16*1.2
    return str(datetime.timedelta(seconds=s))

print(revPred(2, 33, 12))

2 days, 1:01:26.400000


---
<div align="center"><img src="data/MinHamDist.PNG" width="600px"></div>

---
**Problem 2:** Given the optimal 10-base motif from **Problem 1**, find an upper bound on the <code>minHammingDist</code> for the optimal 12-base motif. Explain how you arrived at your answer.

Since we are looking for an upper bound on the minimum Hamming Distance, we want to find the distribution(s) of bases that produce(s) the most disagreement among the 7 motifs (since we have 7 sequences/promoters) given the optimal 10-base motif, and pick the plurality. For the optimal 11-base motif, for example, we can find a plurality of bases by appending each possible base to the end, or the beginning, of the given 10-base optimal motif at least once. Since we have 4 possible bases and 7 sequences/promoters, this means that three of the bases will be appended twice to the optimal motif as follows:

    AGCCTGGGCA + A
    AGCCTGGGCA + C
    AGCCTGGGCA + G
    AGCCTGGGCA + T --> at this point we have appended every possible base;
                       now we keep appending in the same order until we have 7 sequences
    AGCCTGGGCA + A
    AGCCTGGGCA + C
    AGCCTGGGCA + G

Following this method, we can conclude that for any base-appending order there will be three motifs that repeat twice, and a fourth one that is unique. Therefore, we want to the winning motif among the three repeated ones, meaning that the plurality is equal to 2. This means that the minimum Hamming Distance will be 5 (i.e., the motifs not picked) when adding an extra base to the 10-base optimal motif. However, since the question asks about the optimal 12-base motif (i.e., when adding two extra bases):

minHammingDist = 5 (for 11th base) + 5 (for 12th base) + 9 (minHamDist implied in the given 10-base motif) = 19.

---
**Problem 3:** A simple optimization can be applied to <code>**MedianStringMotifSearch()**</code> as follows: If ever during the <code>**ScanAndScoreMotif()**</code> function the Hamming distance <code>(totalDist</code> in <code>**ScanAndScoreMotif()**)</code>, exceeds the smallest Hamming distance seen thus far <code>(minHammingDist</code> in <code>**MedianStringMotifSearch()**)</code>, scanning through subsequent sequences can be terminated early. Implement this strategy and use it to search again for the best 10-base motif. Note that this optimization requires an additional <code>maxTotal</code> argument to be passed to the <code>**ScanAndScoreMotif()**</code>, and therefore both <code>**MedianStringMotifSearch()**</code> and <code>**ScanAndScoreMotif()**</code> require modification. Run the new optimized version, report its run time, and revise the estimate of how long it might take to find an optimal 12-base motif.

In [4]:
def ScanAndScoreMotif(DNA, motif, maxTotal=1000000):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if HammingDist < minHammingDist:
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
        if totalDist > maxTotal:
            break
    return bestAlignment, totalDist

def BetterMedianStringMotifSearch(DNA, k):
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for pattern in itertools.product('ACGT', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif, maxTotal=minHammingDist)
        if dist < minHammingDist:
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

In [5]:
%time BetterMedianStringMotifSearch(dna, 10)

Wall time: 1h 31min 46s


([875, 74, 114, 371, 6, 72, 190], 9, 'AGCCTGGGCA')

In [6]:
print(revPred(1, 31, 46))

1 day, 5:21:55.200000


---
**Problem 4:** Your upper-bound <code>minHammingDist</code> estimate from **Problem 2** can be used to further accelerate <code>**MedianStringMotifSearch()**</code> as follows: It can be used as the intitial setting of the <code>minHammingDist</code> rather than setting it to <code>k\*len(DNA)</code> in addition to the changes made in **Problem 3**. In the cell provided below write <code>**EvenBetterMedianMotifSearch(DNA, k, minHammingDist)**</code>, where the extra argument is used to set your upper-bound prediction.

In [7]:
def ScanAndScoreMotif(DNA, motif, maxTotal=1000000):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if HammingDist < minHammingDist:
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
        if totalDist > maxTotal:
            break
    return bestAlignment, totalDist

def EvenBetterMedianMotifSearch(DNA, k, minHammingDist=-1):
    bestAlignment = []
    kmer = ''

    # Handles the case where no third argument is passed to the function
    # and handles the case where minHammingDist is the best choice
    minHammingDist = k*len(DNA) if minHammingDist < 0 else minHammingDist + 1

    for pattern in itertools.product('ACGT', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif, maxTotal=minHammingDist)
        if dist < minHammingDist:
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

In [8]:
%time EvenBetterMedianMotifSearch(dna, 10, minHammingDist=19)

Wall time: 1h 30min 5s


([875, 74, 114, 371, 6, 72, 190], 9, 'AGCCTGGGCA')

In [9]:
%time EvenBetterMedianMotifSearch(dna, 10, minHammingDist=9)

Wall time: 1h 29min 24s


([875, 74, 114, 371, 6, 72, 190], 9, 'AGCCTGGGCA')

---
**Problem 5:** The cost of generating the upper bound estimate in **Problem 2** was considerable. An alternative upper bound can be found by running <code>**ContainedMotifSearch()**</code>, which considers only *k*-mers that appear in the given sequences as possible motifs. Use this function with *k* = 12 to establish an alternative upper bound.

In [10]:
%time ContainedMotifSearch(dna, 12)

3514 Motifs in our set
Wall time: 34.5 s


([332, 72, 160, 369, 122, 70, 164], 15, 'TGAGCCTGGGCA')